In [1]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
    julia -e '\
        using Pkg                                                          ;\
        Pkg.update()                                                       ;\
        Pkg.add("IJulia")                                                  ;\
        Pkg.add("BenchmarkTools")                                          ;\
        Pkg.add("CuArrays")                                                ;\
        Pkg.add("BenchmarkTools")                                          ;\
        Pkg.add("DataFrames")                                              ;\
        Pkg.add("CSV")                                                     ;\
        Pkg.add("Statistics")                                              ;\
        Pkg.precompile()                                                   ;\'
echo 'Done'

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.ju

## Day 20 作業：將 missing 值替代為其他值

由於現實世界中的資料可能有缺漏，因此資料中就會有 missing 值存在，而在資料科學及機器學習中，缺漏值的處理關係到分析或預測的結果。

在今天的作業中，請將資料集中的年齡缺漏值，依性別分別替換為資料集中男性或女性的年齡平均值。

本次作業使用的資料集：Kaggle Titanic: Machine Learning from Disaster 的 train 資料集

競賽主頁：[https://www.kaggle.com/c/titanic/overview](https://www.kaggle.com/c/titanic/overview)

請自行下載資料集 **train.csv**：[https://www.kaggle.com/c/titanic/data](https://www.kaggle.com/c/titanic/data)

呼叫 `describe()` 函式時，加上 `:nmissing` 參數，顯示各 column 的 missing 值數目。

可以看到 Age 共有 177 個缺漏值。

In [2]:
!julia -e '\
using DataFrames, CSV                                                                                           ;\
url =  "https://julia-fileentity.cupoy.com/1st/homework/example/1584094958965/train.csv?t=1587116099896"        ;\
df = CSV.read(download(url))                                                                                    ;\
println( first(df,5) )                                                                                          ;\
println()                                                                                                       ;\
println( "size of df : ", size(df) )                                                                            ;\
println( describe(df, :nmissing) )                                                                              ;\'

5×12 DataFrame
│ Row │ PassengerId │ Survived │ Pclass │ Name                                                │ Sex    │ Age      │ SibSp │ Parch │ Ticket           │ Fare    │ Cabin   │ Embarked │
│     │ Int64       │ Int64    │ Int64  │ String                                              │ String │ Float64⍰ │ Int64 │ Int64 │ String           │ Float64 │ String⍰ │ String⍰  │
├─────┼─────────────┼──────────┼────────┼─────────────────────────────────────────────────────┼────────┼──────────┼───────┼───────┼──────────────────┼─────────┼─────────┼──────────┤
│ 1   │ 1           │ 0        │ 3      │ Braund, Mr. Owen Harris                             │ male   │ 22.0     │ 1     │ 0     │ A/5 21171        │ 7.25    │ missing │ S        │
│ 2   │ 2           │ 1        │ 1      │ Cumings, Mrs. John Bradley (Florence Briggs Thayer) │ female │ 38.0     │ 1     │ 0     │ PC 17599         │ 71.2833 │ C85     │ C        │
│ 3   │ 3           │ 1        │ 3      │ Heikkinen, Miss. Laina           

計算男性及女性的平均年齡。

【提示】使用 `skipmissing()` 去掉缺漏值後，再計算平均值。

In [3]:
!julia -e '\
using DataFrames, CSV, Statistics                                                                               ;\
url =  "https://julia-fileentity.cupoy.com/1st/homework/example/1584094958965/train.csv?t=1587116099896"        ;\
df = CSV.read(download(url))                                                                                    ;\
x = female_age_avg = df[df.Sex .== "female", :Age ]                                                             ;\
y = male_age_avg = df[df.Sex .== "male", :Age ]                                                                 ;\
println( "女性的平均年齡  ", mean(skipmissing(x)) )                                                              ;\
println( "男性的平均年齡  ", mean(skipmissing(y)) )                                                              ;\'

女性的平均年齡  27.915708812260537
男性的平均年齡  30.72664459161148


將 Age column 中的 missing 值，依性別替換為上面計算出來的平均年齡值。

【提示】可產生新的 column 來存放新的結果，而不取代掉原先的 Age column。

In [4]:
!julia -e '\
using DataFrames, CSV, Statistics                                                                               ;\
url =  "https://julia-fileentity.cupoy.com/1st/homework/example/1584094958965/train.csv?t=1587116099896"        ;\
df = CSV.read(download(url))                                                                                    ;\
x = female_age_avg = df[df.Sex .== "female", :Age ]                                                             ;\
newX = coalesce.(x, mean(skipmissing(x)))                                                                       ;\
println( "original female_age")                                                                                 ;\
println( x )                                                                                                    ;\
println( "new female_age" )                                                                                     ;\
println( newX )                                                                                                 ;\
y = male_age_avg = df[df.Sex .== "male", :Age ]                                                                 ;\
newY = coalesce.(y , mean(skipmissing(y )))                                                                     ;\
println( "original male_age")                                                                                   ;\
println( y )                                                                                                    ;\
println( "new male_age" )                                                                                       ;\
println( newY )                                                                                                 ;\'

original female_age
Union{Missing, Float64}[38.0, 26.0, 35.0, 27.0, 14.0, 4.0, 58.0, 14.0, 55.0, 31.0, missing, 15.0, 8.0, 38.0, missing, missing, missing, 18.0, 14.0, 40.0, 27.0, 3.0, 19.0, missing, 18.0, 49.0, 29.0, 21.0, 5.0, 38.0, 29.0, 17.0, 16.0, 30.0, missing, 17.0, 33.0, 23.0, 34.0, 28.0, 21.0, missing, 14.5, 20.0, 17.0, 2.0, 32.5, missing, 47.0, 29.0, 19.0, missing, 22.0, 24.0, 9.0, 22.0, 16.0, 40.0, missing, 45.0, 1.0, 50.0, missing, 4.0, missing, 32.0, 19.0, 44.0, 58.0, missing, 24.0, 2.0, 16.0, 35.0, 31.0, 27.0, 32.0, missing, 35.0, 5.0, missing, 8.0, missing, missing, 25.0, 24.0, 29.0, 41.0, 29.0, missing, 30.0, 35.0, 50.0, missing, 58.0, 35.0, 41.0, missing, 63.0, 45.0, 35.0, 22.0, 26.0, 19.0, 24.0, 2.0, 50.0, missing, missing, missing, 17.0, 30.0, 24.0, 18.0, 26.0, 26.0, 24.0, 31.0, 40.0, 30.0, 22.0, 36.0, 36.0, 31.0, 16.0, missing, missing, 41.0, 24.0, 24.0, 40.0, missing, 22.0, 38.0, missing, missing, 45.0, 60.0, missing, missing, 24.0, 3.0, missing, 22.0, 42.0, 1.0, 3